# Meta Prompting

Meta prompting is an advanced technique in prompt engineering that emphasizes the structural and syntactical organization of tasks and problems rather than focusing on their specific content. The objective is to create a more abstract, form-driven way of engaging with large language models (LLMs), highlighting patterns and structure over traditional content-focused methods.

As outlined by [Zhang et al. (2024)](https://arxiv.org/abs/2311.11482), the defining features of meta prompting include:

* Structure-Oriented: Prioritizes the organization and pattern of problems and solutions instead of specific content.
* Syntax-Guided: Leverages syntax as a template to shape the expected responses or solutions.
* Abstract Frameworks: Uses abstract examples as blueprints, demonstrating the structure of tasks without relying on concrete details.
* Domain Versatility: Can be applied across multiple fields, offering structured solutions to diverse problem types.
* Categorical Approach: Draws on type theory to organize and categorize components logically, enhancing prompt coherence and precision.

## Running this code on MyBind.org

Note: remember that you will need to **adjust CONFIG** with **proper URL and API_KEY**!

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/GenILab-FAU/prompt-eng/HEAD?urlpath=%2Fdoc%2Ftree%2Fprompt-eng%2Fmeta.ipynb)



In [2]:
############################################################
## META ANALYSIS: COMPARE TECHNIQUES & OUTPUTS
############################################################

import os
import pandas as pd
from _pipeline import create_payload, model_req

# 1) Paths to your log files
ZERO_SHOT_LOG = "data/zero_shot_logs.csv"
FEW_SHOT_LOG  = "data/few_shot_logs.csv"
COT_LOG       = "data/chain_of_thought_logs.csv"
SC_LOG        = "data/self_consistency_logs.csv"

def load_logs(log_path):
    """Load CSV logs if the file exists, otherwise return an empty DataFrame."""
    if os.path.exists(log_path):
        return pd.read_csv(log_path)
    else:
        return pd.DataFrame()

# 2) Load partial outputs from each technique
df_zero = load_logs(ZERO_SHOT_LOG)
df_few  = load_logs(FEW_SHOT_LOG)
df_cot  = load_logs(COT_LOG)
df_sc   = load_logs(SC_LOG)

# 3) Function to extract a representative or final sample output
def get_sample_output(df, technique_name):
    """
    Returns a short label plus a snippet of the technique's output.
    For demonstration, we grab the last row's 'response' or 'final_output' column.
    """
    if df.empty:
        return f"**{technique_name}:** No logs found.\n"
    row = df.iloc[-1]
    output_text = row.get("final_output") or row.get("response", "No output")
    # Truncate the output for the meta prompt if it's too long
    snippet = (output_text[:400] + "...") if len(output_text) > 400 else output_text
    return f"**{technique_name}** (Sample):\n{snippet}\n"

# 4) Build a meta-analysis prompt
zero_snippet = get_sample_output(df_zero, "Zero-Shot")
few_snippet  = get_sample_output(df_few,  "Few-Shot")
cot_snippet  = get_sample_output(df_cot,  "Chain-of-Thought")
sc_snippet   = get_sample_output(df_sc,   "Self-Consistency")

META_PROMPT = f"""
You are an AI researcher analyzing various prompt-engineering techniques:
- Zero-Shot
- Few-Shot
- Chain-of-Thought
- Self-Consistency

Below are representative outputs from each technique:

{zero_snippet}

{few_snippet}

{cot_snippet}

{sc_snippet}

Please compare these approaches based on:
1) Clarity and structure
2) Accuracy or completeness
3) Speed or efficiency (if known)
4) Overall quality

Then provide a brief meta-analysis explaining which technique(s) might be best for
generating a 'Project Overview' and why. Suggest potential improvements.
"""

# 5) Create the payload for meta analysis
payload_meta = create_payload(
    target="open-webui",
    model="qwen2",  # Example model for meta analysis
    prompt=META_PROMPT,
    temperature=0.7,
    num_ctx=250,
    num_predict=300
)

# 6) Send the request
time_meta, response_meta = model_req(payload=payload_meta)

# 7) Print the meta-analysis
print("===== META ANALYSIS OUTPUT =====")
print(response_meta)
if time_meta:
    print(f"\nTime taken for meta analysis: {time_meta}s")

# 8) (Optional) Save meta analysis to file
os.makedirs("outputs", exist_ok=True)
with open("outputs/meta_analysis.md", "w", encoding="utf-8") as f:
    f.write(response_meta)


ModuleNotFoundError: No module named 'pandas'